In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import os
from ipywidgets import interact, interact_manual
from mpl_toolkits.axes_grid1 import make_axes_locatable

path = os.path.join(os.getcwd(), 'Documents', 'GitHub', 'final-project-oiivia-ashu')
shape_file = os.path.join(path, 'Municipality.shp') 
# Source: https://hub-cookcountyil.opendata.arcgis.com/datasets/534226c6b1034985aca1e14a2eb234af_2?selectedAttribute=MUNICIPALITY 

# Importing and merging datasets
municipalities = geopandas.read_file(shape_file)
class_summary = pd.read_csv(os.path.join(path, 'class_summary.csv'))
spatial_data = municipalities.merge(class_summary, how = 'left', on = ['MUNICIPALI']) # EPSG: 3435

In [2]:
# Initialization and type conversions
def clean_up(df):
   
    df['share'] = 0
    df['YEAR'] = df['YEAR'].astype('Int64')
    years = list(range(2012,2018))
    categories = list(df.DISPOSITION_CHARGED_CLASS.dropna().unique())
    df['DISPOSITION_CHARGED_CLASS'] = df['DISPOSITION_CHARGED_CLASS'].astype(str)
    cities = list(df.MUNICIPALI.dropna().unique())
    return df, years, categories, cities

In [3]:
def summarize(df, years, categories, cities):
        
    df = df[df['DISPOSITION_CHARGED_CLASS'].notna()]
    df = df[df['YEAR'].notna()]
    
    for year in years:
        for c in categories:
            for city in cities:
                d = len(df[(df['YEAR'] == year) & (df['DISPOSITION_CHARGED_CLASS'] == c)])
                n = len(df[(df['YEAR'] == year) & (df['DISPOSITION_CHARGED_CLASS'] == c) & (df['MUNICIPALI'] == city)])
                if d != 0:
                    df.loc[(df.YEAR == year) & (df.DISPOSITION_CHARGED_CLASS == c) & (df.MUNICIPALI == city), 'share'] = 100*n/d
          
    return(df)

In [4]:
def plotter(felony):
    
    fig, ax = plt.subplots(figsize=(8,8))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('left', size='5%', pad=0.5)
    ax = spatial_data[(spatial_data['DISPOSITION_CHARGED_CLASS'] == felony)].plot(ax=ax, column='share', legend=True, cax=cax)
    ax.axis('off')
    ax.set_title('Felonies in Cook County');
    return ax

In [5]:
spatial_data, years, categories, cities = clean_up(spatial_data)
spatial_data = summarize(spatial_data, years, categories, cities)

In [6]:
@interact(felony = categories)
def make_plot_for(felony = 'C'):
     plot = plotter(felony)

interactive(children=(Dropdown(description='felony', index=7, options=('1', '2', '3', '4', 'A', 'M', 'X', 'C',…